<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/develop/Fine_Tuning_Deberta_Token_Classification%5BNER%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn
!pip install seqeval[gpu]

In [ ]:
%%capture
! pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
! pip install git+https://github.com/huggingface/accelerate

In [47]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,

In [29]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [30]:
!nvidia-smi

Wed Jul 13 04:36:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load the dataset

In [31]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2003")

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

In [33]:
raw_datasets['train'][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [34]:
raw_datasets['train'].features['ner_tags'],raw_datasets['train'].features['ner_tags'].feature.names

(Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None),
 ['O',
  'B-PER',
  'I-PER',
  'B-ORG',
  'I-ORG',
  'B-LOC',
  'I-LOC',
  'B-MISC',
  'I-MISC'])

In [35]:
label_names = raw_datasets['train'].features['ner_tags'].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [36]:
sample_tokens = raw_datasets['train'][0]['tokens']
sample_tags = raw_datasets['train'][0]['ner_tags']
sample_tokens,sample_tags
s=""
y=""
for tokens,tags in zip(sample_tokens,sample_tags):
  s+=tokens+" "
  y+=label_names[tags]+" "
s,y

('EU rejects German call to boycott British lamb . ',
 'B-ORG O B-MISC O O O B-MISC O O ')

In [51]:
#checkpoint_name = "microsoft/deberta-v3-base"
checkpoint_name = "bert-base-cased"
tokeinzer = AutoTokenizer.from_pretrained(checkpoint_name)
tokeinzer.is_fast

True

In [52]:
tokenized_text = tokeinzer(raw_datasets['train'][0]['tokens'],is_split_into_words=True)
print(tokenized_text)
print(tokenized_text.tokens())
print(tokeinzer.decode(tokenized_text['input_ids']))
print(tokenized_text.word_ids())

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[CLS] EU rejects German call to boycott British lamb. [SEP]
[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


In [53]:
sample_tags,tokenized_text.word_ids()

([3, 0, 7, 0, 0, 0, 7, 0, 0], [None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None])

In [54]:
new_labels =[]
for word_id in tokenized_text.word_ids():
  if word_id is None:
    new_labels.append(-100)
  else:
    new_labels.append(sample_tags[word_id])
new_labels

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [55]:
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [56]:
def align_labels_with_tokens(labels,word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id!=current_word:
      #start of a new token
      current_word=word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
      new_labels.append(-100)
    else:
      # same word as previous token
      label = labels[word_id]
      if label %2 ==1: # all B- tokens are in odd position, I- tokens are in even position. So If the label is B-XXX we change it to I-XXX
        label+=1
      new_labels.append(label)
  return new_labels

In [57]:
align_labels_with_tokens(sample_tags,tokenized_text.word_ids())

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [59]:
def tokenize_and_align_labels(batch_of_samples):
  batch_encoding = tokeinzer(batch_of_samples['tokens'],is_split_into_words=True,truncation=True,padding="max_length",max_length=256)
  batch_ner_tags = batch_of_samples['ner_tags']
  new_labels = []
  for i,labels in enumerate(batch_ner_tags):
    word_ids = batch_encoding.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels,word_ids))
  batch_encoding['labels'] = new_labels
  return batch_encoding

In [60]:
tokenizd_dataset = raw_datasets.map(tokenize_and_align_labels,batched=True,remove_columns = raw_datasets["train"].column_names)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [61]:
tokenizd_dataset,tokenizd_dataset['train'][0],len(tokenizd_dataset['train']['input_ids'])

(DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3454
    })
}),
 {'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

##Fine Tuning


In [64]:
## metrics
from datasets import load_metric
metric = load_metric("seqeval")

This metric does not behave like the standard accuracy: it will actually take the lists of labels as strings, not integers, so we will need to fully decode the predictions and labels before passing them to the metric. Let’s see how it works. First, we’ll get the labels for our first training example:

In [70]:
labels = raw_datasets["train"][0:3]["ner_tags"]
labels

[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]]

In [73]:
label_names
labels = [[label_names[l] for l in label] for label in labels]
labels

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O']]

In [75]:
preds = labels.copy()
preds[0][2] = "O"
preds

[['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O']]

In [69]:
metric.compute??

In [76]:
metric.compute(predictions=[preds], references=[labels])

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-PER', 'I-PER'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-LOC', 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{"LOC', 'O']": {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 "ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']": {'f1': 1.0,
  'number': 1,
  'precision': 1.0,
  'recall': 1.0},
 "PER', 'I-PER']": {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [78]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [80]:
id2label = {str(i):label for i,label in enumerate(label_names)}
label2id = {label:str(i) for i,label in enumerate(label_names)}
id2label,label2id

({'0': 'O',
  '1': 'B-PER',
  '2': 'I-PER',
  '3': 'B-ORG',
  '4': 'I-ORG',
  '5': 'B-LOC',
  '6': 'I-LOC',
  '7': 'B-MISC',
  '8': 'I-MISC'},
 {'B-LOC': '5',
  'B-MISC': '7',
  'B-ORG': '3',
  'B-PER': '1',
  'I-LOC': '6',
  'I-MISC': '8',
  'I-ORG': '4',
  'I-PER': '2',
  'O': '0'})

In [103]:
from torch.utils.data import Dataset
import torch
class NERDataset(Dataset):
  def __init__(self,tokenized_dataset):
    self.data = tokenized_dataset
  def __getitem__(self, idx):
        print(idx)
        item = {key: torch.tensor(val) for key, val in self.data[idx].items()}
        return item
  def __len__(self):
        return len(self.data)


In [97]:
tokenizd_dataset['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [111]:
train_dataset = NERDataset(tokenizd_dataset['train'])
validation_dataset = NERDataset(tokenizd_dataset['validation'])
test_dataset = NERDataset(tokenizd_dataset['test'])

In [110]:
train_dataset[1:3]['input_ids'].shape

slice(1, 3, None)


torch.Size([2, 256])

In [106]:
next(iter(train_dataset))

0


{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102

In [114]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint_name,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [123]:
model.config.num_labels

9

In [120]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

check whether the data is created correctly by fetching the first batch from the data loader, and then printing out the shapes of the tensors, decoding the input_ids back to text, etc.

In [122]:
for k,v in next(iter(train_dataloader)).items():
    print(k, v.shape)

8914
1923
5365
11220
7279
7089
5314
501
input_ids torch.Size([8, 256])
token_type_ids torch.Size([8, 256])
attention_mask torch.Size([8, 256])
labels torch.Size([8, 256])


In [124]:
def training_function():
  model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis